# Quantum Winter 2020 | TriQ | Q_4

## Objectives:
1. Store 41*41 array in 11 qubit QRAM  
2. Reuse leftover addresses from Matrix A to store Matrix U  
3. Solve Linear equation using Spectral Method as per https://arxiv.org/pdf/1901.00961.pdf
4. Evaluate Wall time against classical implementation
5. Error mitigation in implementation of PDE_Oracle

### Update as of 17-12-2020
- Only Objective 1 achieved. Program Incomplete.

In [97]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ,QuantumRegister, ClassicalRegister  
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from iqx import *
import numpy as np


# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2020-12-17 03:00:58,616: Credentials are already in use. The existing account in the session will be replaced.


In [98]:
#Define_State_values
def zero(qc,data,address):
    return qc,data,address
def one(qc,data,address):
    qc.mct(address,data[0])
    return qc,data,address
def two(qc,data,address):
    qc.mct(address,data[1])
    return qc,data,address
def three(qc,data,address):
    qc.mct(address,data[0])
    qc.mct(address,data[1])
    return qc,data,address
def minus_one(qc,data,address):
    qc.mct(address,data[2])
    return qc,data,address
def minus_two(qc,data,address):
    qc.mct(address,data[0])
    qc.mct(address,data[2])
    return qc,data,address
def minus_three(qc,data,address):
    qc.mct(address,data[1])
    qc.mct(address,data[2])
    return qc,data,address

#Define State_assign_function
def States(val,data,qc,address):
    if (val == 0.0):
        return zero(qc,data,address)
    if (val == 1.0):
        return one(qc,data,address)
    if (val == 2.0):
        return two(qc,data,address)
    if (val == 3.0):
        return three(qc,data,address)
    if (val == -1.0):
        return minus_one(qc,data,address)
    if (val == -2.0):
        return minus_two(qc,data,address)
    if (val == -1.0):
        return minus_three(qc,data,address)

#Data_States_as_phase_vectors ***EXPERIMENTAL***
"""def zero(qc,data):
    return qc,data
def one(qc,data):
    qc.x(data)
    return qc,data
def two(qc,data):
    qc.y(data)
    qc.x(data)
    return qc,data
def three(qc,data):
    qc.y(data)
    return qc,data
def minus_one(qc,data):
    qc.y(data)
    qc.z(data)
    qc.y(data)
    return qc,data
def minus_two(qc,data):
    qc.y(data)
    qc.z(data)
    qc.x(data)
    return qc,data
def minus_three(qc,data):
    qc.y(data)
    qc.x(data)
    qc.y(data)
    return qc,data

#Reset Data_States
def reset_zero(qc,data):
    qc.barrier()
    return qc,data
def reset_one(qc,data):
    qc.x(data)
    qc.barrier()
    return qc,data
def reset_two(qc,data):
    qc.x(data)
    qc.y(data)
    qc.barrier()
    return qc,data
def reset_three(qc,data):
    qc.y(data)
    qc.barrier()
    return qc,data
def reset_minus_one(qc,data):
    qc.y(data)
    qc.z(data)
    qc.y(data)
    qc.barrier()
    return qc,data
def reset_minus_two(qc,data):
    qc.x(data)
    qc.z(data)
    qc.y(data)
    qc.barrier()
    return qc,data
def reset_minus_three(qc,data):
    qc.y(data)
    qc.x(data)
    qc.y(data)
    qc.barrier()
    return qc,data
    
def Reset_States(val,data,qc):
    if (val == 0):
        return reset_zero(qc,data)
    if (val == 1):
        return reset_one(qc,data)
    if (val == 2):
        return reset_two(qc,data)
    if (val == 3):
        return reset_three(qc,data)
    if (val == -1):
        return reset_minus_one(qc,data)
    if (val == -2):
        return reset_minus_two(qc,data)
    if (val == -1):
        return reset_minus_three(qc,data)
"""

'def zero(qc,data):\n    return qc,data\ndef one(qc,data):\n    qc.x(data)\n    return qc,data\ndef two(qc,data):\n    qc.y(data)\n    qc.x(data)\n    return qc,data\ndef three(qc,data):\n    qc.y(data)\n    return qc,data\ndef minus_one(qc,data):\n    qc.y(data)\n    qc.z(data)\n    qc.y(data)\n    return qc,data\ndef minus_two(qc,data):\n    qc.y(data)\n    qc.z(data)\n    qc.x(data)\n    return qc,data\ndef minus_three(qc,data):\n    qc.y(data)\n    qc.x(data)\n    qc.y(data)\n    return qc,data\n\n#Reset Data_States\ndef reset_zero(qc,data):\n    qc.barrier()\n    return qc,data\ndef reset_one(qc,data):\n    qc.x(data)\n    qc.barrier()\n    return qc,data\ndef reset_two(qc,data):\n    qc.x(data)\n    qc.y(data)\n    qc.barrier()\n    return qc,data\ndef reset_three(qc,data):\n    qc.y(data)\n    qc.barrier()\n    return qc,data\ndef reset_minus_one(qc,data):\n    qc.y(data)\n    qc.z(data)\n    qc.y(data)\n    qc.barrier()\n    return qc,data\ndef reset_minus_two(qc,data):\n    qc

In [102]:
#Q_RAM (Max_Capacity = 2048 States @ 11 Qubit Address_Registers, 3 Data_Registers)
address = QuantumRegister(11, name = "address")
data = QuantumRegister(3, name = "data")
c = ClassicalRegister(11,name = "measurement")
qc = QuantumCircuit(address,data,c)

qc.h(address)
qc.barrier()

aa = np.ones(38) #central diag
bb = 1.0*np.ones(39) #upper_t diag
cc = -1.0*np.ones(38) #lower_t diag
A = np.diag(aa,-1) + np.diag(bb,0) + np.diag(cc,1) #Matrix A acc to hint
A[0,0]=3.0 #a41
A[-1,-1]=1 #b1
A[0,1]=-2.0 #c41

print(A) #debugging

def QRAM_11(A):
    for i in range(39): #Store a 41*41 Matrix
        for j in range(39):
            binary = f'{((i*10)+j):00000000011b}'
            if(binary[0] == '0'):
                qc.x(address[0])
            if(binary[1] == '0'):
                qc.x(address[1])
            if(binary[2] == '0'):
                qc.x(address[2])
            if(binary[3] == '0'):
                qc.x(address[3])
            if(binary[4] == '0'):
                qc.x(address[4])
            if(binary[5] == '0'):
                qc.x(address[5])
            if(binary[6] == '0'):
                qc.x(address[6])
            if(binary[7] == '0'):
                qc.x(address[7])
            if(binary[8] == '0'):
                qc.x(address[8])
            if(binary[8] == '0'):
                qc.x(address[8])
            if(binary[9] == '0'):
                qc.x(address[9])
            if(binary[10] == '0'):
                qc.x(address[10])

            States(A[i,j],data,qc,address)

            if(binary[0] == '0'):
                qc.x(address[0])
            if(binary[1] == '0'):
                qc.x(address[1])
            if(binary[2] == '0'):
                qc.x(address[2])
            if(binary[3] == '0'):
                qc.x(address[3])
            if(binary[4] == '0'):
                qc.x(address[4])
            if(binary[5] == '0'):
                qc.x(address[5])
            if(binary[6] == '0'):
                qc.x(address[6])
            if(binary[7] == '0'):
                qc.x(address[7])
            if(binary[8] == '0'):
                qc.x(address[8])
            if(binary[8] == '0'):
                qc.x(address[8])
            if(binary[9] == '0'):
                qc.x(address[9])
            if(binary[10] == '0'):
                qc.x(address[10])
                
            qc.barrier()

#call functions
QRAM_11(A)
#Oracle_PDE()
#Oracle_2_PDE()
#Counter_Oracle_2_PDE()
#Counter_Oracle_PDE()
QRAM_11(A)
#Compare_PDE()

qc.barrier()

qc = qc.reverse_bits()
qc.measure(address,c)




#backend = provider.get_backend('ibmq_qasm_simulator')
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend=backend, shots=8000, seed_simulator=12345, backend_options={"fusion_enable":True})
#job = execute(qc, backend=backend, shots=8192)
result = job.result()
count =result.get_counts()
print(count)

qc.draw()

[[ 3. -2.  0. ...  0.  0.  0.]
 [ 1.  1. -1. ...  0.  0.  0.]
 [ 0.  1.  1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  1.  1. -1.]
 [ 0.  0.  0. ...  0.  1.  1.]]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:97: DeprecationWarning: Using `backend_options` kwarg has been deprecated as of qiskit-aer 0.7.0 and will be removed no earlier than 3 months from that release date. Runtime backend options should now be added directly using kwargs for each option.


{'00000000000': 2, '00000000001': 4, '00000010000': 3, '00100000000': 7, '00100000001': 1, '00100000010': 5, '00100000011': 8, '00100000100': 4, '00100000101': 4, '00100000110': 8, '00100000111': 2, '00100001000': 2, '00100001001': 5, '00100001010': 7, '00100001011': 2, '00100001100': 5, '00100001101': 3, '00100001110': 3, '00100001111': 2, '00000010001': 3, '00100010000': 4, '00100010001': 4, '00100010010': 3, '00100010011': 6, '00100010100': 3, '00100010101': 6, '00100010110': 3, '00100010111': 1, '00100011000': 7, '00100011001': 2, '00100011010': 4, '00100011011': 5, '00100011100': 5, '00100011101': 6, '00100011110': 4, '00100011111': 6, '00000010010': 6, '00100100000': 3, '00100100001': 5, '00100100010': 3, '00100100011': 9, '00100100100': 3, '00100100101': 6, '00100100110': 2, '00100100111': 3, '00100101001': 4, '00100101010': 2, '00100101011': 1, '00100101100': 3, '00100101101': 6, '00100101110': 7, '00100101111': 5, '00000010011': 5, '00100110000': 2, '00100110001': 4, '00100110

ValueError: Image size of 1710x478674 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1710.73x478674 with 1 Axes>